<a href="https://colab.research.google.com/github/nerumur/Valeria_ML/blob/main/RNN_Val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dm-sonnet -q

In [3]:
import re
import nltk
import numpy as np
import pandas as pd
import sonnet as snt
import tensorflow as tf

from nltk.corpus import stopwords

nltk.download('stopwords')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv('/content/Corona.csv', encoding='latin-1')   # подумала, что будет интересно погенерить твиты про коронавирус
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [5]:
df = df[['OriginalTweet']][:5000].dropna()   # нам нужна только 1 колонка с твитами, возьмем 5к твитов (иначе сессия крашится)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  5000 non-null   object
dtypes: object(1)
memory usage: 39.2+ KB


In [6]:
stop_words = stopwords.words('english')      # чистим

def data_cleaner(tweet):
    tweet = re.sub(r'http\S+', ' ', tweet)   # url
    tweet = re.sub(r'<.*?>',' ', tweet)      # html
    tweet = re.sub(r'\d+',' ', tweet)        # цифры
    tweet = re.sub(r'#\w+',' ', tweet)       # хештеги
    tweet = re.sub(r'@\w+',' ', tweet)       # @
    tweet = tweet.split()                    # стоп-слова
    tweet = " ".join([word for word in tweet if not word in stop_words])
    return tweet

df['OriginalTweet'] = df['OriginalTweet'].apply(data_cleaner)
df.head()

,OriginalTweet
0,
1,advice Talk neighbours family exchange phone n...
2,Coronavirus Australia: Woolworths give elderly...
3,"My food stock one empty... PLEASE, panic, THER..."
4,"Me, ready go supermarket outbreak. Not I'm par..."


In [7]:
# Инициализируем токенизатор
tokenizer = Tokenizer()

# Обучаем токенизатор на заголовках
tokenizer.fit_on_texts(df['OriginalTweet'])

# Преобразуем заголовки в последовательности чисел
sequences = tokenizer.texts_to_sequences(df['OriginalTweet'])

# Создаем входные и выходные данные
X = []
y = []
for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])

Посмотрим на наши вектора

**Вопрос:** что кодируют эти числа? Какие значения попадают в y, а что хранится в X'ах?

X — входные последовательности, y —  целевые значения (нужно предсказать на основе X). Похоже на снежный ком.

In [8]:
X[:10], y[:10]

([[477],
  [477, 681],
  [477, 681, 1888],
  [477, 681, 1888, 179],
  [477, 681, 1888, 179, 2352],
  [477, 681, 1888, 179, 2352, 1046],
  [477, 681, 1888, 179, 2352, 1046, 1470],
  [477, 681, 1888, 179, 2352, 1046, 1470, 801],
  [477, 681, 1888, 179, 2352, 1046, 1470, 801, 354],
  [477, 681, 1888, 179, 2352, 1046, 1470, 801, 354, 257]],
 [681, 1888, 179, 2352, 1046, 1470, 801, 354, 257, 1046])

##### Создание модели

In [9]:
# Определяем класс RNN, который наследуется от snt.RNNCore (базовый класс для RNN-ячеек в Sonnet)
class RNN(snt.RNNCore):

  # Конструктор класса, инициализирует параметры RNN
  def __init__(self, hidden_size, activation=tf.tanh, name="vanilla_rnn"):
    """
    hidden_size: размер скрытого слоя
    activation: тип функции активации
    name: название модели
    """
    # Вызываем конструктор родительского класса (snt.RNNCore)
    super(RNN, self).__init__(name=name)

    # Сохраняем размер скрытого состояния (количество нейронов в скрытом слое)
    self._hidden_size = hidden_size

    # Сохраняем функцию активации (по умолчанию используется гиперболический тангенс)
    self._activation = activation


  # Метод _build определяет вычисления, которые происходят в RNN на каждом шаге
  def _build(self, input_, prev_state):
    """
    input_: тензор с текущим x_t
    prev_tate: тензор с h_{t-1}
    """
    # Создаем линейный слой для преобразования входных данных в скрытое состояние
    self._in_to_hidden_linear = snt.Linear(
        self._hidden_size, name="in_to_hidden")

    # Создаем линейный слой для преобразования предыдущего скрытого состояния в новое
    self._hidden_to_hidden_linear = snt.Linear(
        self._hidden_size, name="hidden_to_hidden")

    # Применяем линейное преобразование к входным данным
    in_to_hidden = self._in_to_hidden_linear(input_)

    # Применяем линейное преобразование к предыдущему скрытому состоянию
    hidden_to_hidden = self._hidden_to_hidden_linear(prev_state)

    # Складываем результаты и применяем функцию активации
    output = self._activation(in_to_hidden + hidden_to_hidden)

    # Возвращаем выходное значение и новое скрытое состояние (они одинаковы в этой реализации)
    return output, output

  # Свойство state_size возвращает размерность скрытого состояния
  @property
  def state_size(self):
    return tf.TensorShape([self._hidden_size])

  # Свойство output_size возвращает размерность выходного значения
  @property
  def output_size(self):
    return tf.TensorShape([self._hidden_size])

In [10]:
# Преобразуем списки в массивы numpy
X = np.asarray(X, dtype="object")
y = np.array(y)

# Дополняем последовательности до одинаковой длины
X = pad_sequences(X)

# Преобразуем y в one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

##### Обучение модели

In [11]:
# Создаем модель
model = Sequential()

# Добавляем слой Embedding
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))

# Добавляем слой LSTM
model.add(LSTM(150, return_sequences=False))

# Добавляем полносвязный слой
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Компилируем модель
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Выводим информацию о модели
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Обучаем модель
history = model.fit(X, y, epochs=10, batch_size=64, validation_split=0.2)  # оставляем 10 эпох, иначе сессия крашится

Epoch 1/10
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.0172 - loss: 8.1361 - val_accuracy: 0.0311 - val_loss: 7.8885
Epoch 2/10
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 17s 13ms/step - accuracy: 0.0359 - loss: 7.4830 - val_accuracy: 0.0419 - val_loss: 7.8509
Epoch 3/10
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.0508 - loss: 7.1258 - val_accuracy: 0.0518 - val_loss: 7.8571
Epoch 4/10
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - accuracy: 0.0648 - loss: 6.7424 - val_accuracy: 0.0591 - val_loss: 7.9031
Epoch 5/10
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.0812 - loss: 6.3230 - val_accuracy: 0.0610 - val_loss: 8.0059
Epoch 6/10
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.0983 - loss: 5.8918 - val_accuracy: 0.0640 - val_loss: 8.1453
Epoch 7/10
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - accuracy: 0.1246 - loss: 5.4932 - val_accuracy: 0.0659 - val_loss: 8.2780
Epoch 8/10
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.1647 -

Обновим саммари модели

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 49, 100)        │     1,142,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 11421)          │     1,724,571 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,051,815 (34.53 MB)

 Trainable params: 3,017,271 (11.51 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,034,544 (23.02 MB)

Тестируем

In [19]:
# Функция для генерации текста
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Генерируем новый твит
generated_text = generate_text("Generated tweet:", 10, X.shape[1])
print(generated_text)

generated_text_2 = generate_text("Supermarkets", 5, X.shape[1])
print(generated_text_2)

generated_text_3 = generate_text("Coronavirus", 7, X.shape[1])
print(generated_text_3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Generated tweet: covid panic buying food shortages amid covid experts say people
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Supermarkets online shopping online shopping slots
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Coronavirus outbreak changing co

Сохраняем

In [20]:
# Сохраняем модель
model.save('corona_tweets_generator.keras')